# Prepare data for analysis

Data cleansing, conversions and aggregations preparing the datasets for further analysis.

In [44]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

from datetime import datetime

try:
    sc = SparkContext('local[*]')
    sc.setLogLevel("OFF")
    
    spark = SparkSession(sc)
except ValueError:
    # Prevent the notebook to crash if this block is executed more then once
    pass

import re
import glob
import functools

## OUI file

Parse IEEE Organizationally unique identifier (OUI) file that uniquely identifies a vendors based on the MAC address prefix.

In [104]:
# From: https://www.gsmarena.com/makers.php3
# .main-makers table
# ((<tr>)?<td><a (.+?)>|</a></td>(</tr>)?)
# (<br><span>|</span>)
makers = spark.read.csv("./data/makers.csv", header=True, inferSchema=True)

makers.orderBy(desc("devices")).toPandas().head(10)

,maker,devices
0,Samsung,1133
1,LG,602
2,Nokia,459
3,Motorola,454
4,alcatel,376
5,Micromax,276
6,Huawei,269
7,BLU,262
8,HTC,255
9,ZTE,240


In [50]:
oui_parser = re.compile(r"(?P<prefix>([0-9A-F]{2}-?){3})\s{1,}\(hex\)\t{2}(?P<organization>.*)")

def tupled(txt):
    m = oui_parser.search(txt)
    p, o = (m.group("prefix").replace("-", ":"), m.group("organization"))
    return (p, o)
    
oui = (
    sc.textFile("./data/oui.txt")
    .filter(lambda x: oui_parser.search(x))
    .map(tupled)
    .toDF().toDF("prefix", "organization")
)

(
    oui
    .groupBy("organization")
    .count()
    .orderBy(desc("count"))
).toPandas().head(20)

,organization,count
0,"Cisco Systems, Inc",802
1,"Apple, Inc.",631
2,"Samsung Electronics Co.,Ltd",474
3,"HUAWEI TECHNOLOGIES CO.,LTD",426
4,"ARRIS Group, Inc.",276
5,Intel Corporate,226
6,Texas Instruments,173
7,zte corporation,147
8,Hewlett Packard,140
9,"Hon Hai Precision Ind. Co.,Ltd.",129


In [208]:
import sys

# makers_oui = 

stop_words = ["", "inc", "co", "ltd", "coltd", "llc", "oo", "oy", "cisco", "shenzhen", "electromechanics"]


msx = makers.withColumn("soundex", soundex("maker")).drop("devices")

osx = (    
    oui
    .withColumn("bits", split("organization", " "))
    .select(col("prefix"), col("organization"), explode("bits").alias("bit"))
    .withColumn("safeBit", lower(regexp_replace("bit", "[^A-Za-z0-9]", "")))
    .filter(~col("safeBit").isin(*stop_words))
    .withColumn("soundex", soundex("safeBit"))
    .drop("bit")
)

makers_oui = (
    osx
    .join(msx, msx.soundex == osx.soundex)
    .withColumn("distance", levenshtein("safeBit", lower(col("maker"))))
    .filter(col("distance") == 0)
    .drop("soundex", "safeBit", "distance")
    .distinct()
)

makers_oui.groupBy("organization").count().orderBy(desc("count")).toPandas().head(30)

,organization,count
0,"Apple, Inc.",631
1,"Samsung Electronics Co.,Ltd",474
2,"HUAWEI TECHNOLOGIES CO.,LTD",426
3,zte corporation,147
4,Dell Inc.,101
5,Nokia Corporation,90
6,"Motorola Mobility LLC, a Lenovo Company",80
7,Nokia Danmark A/S,72
8,LG Electronics (Mobile Communications),65
9,Sony Mobile Communications Inc,60


## WiFi logs

Filter and concatenate all data from WiFi logs

In [16]:
mac_address = re.compile(r"([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})")

cols = ['timestamp', 'level', 'mac', 'ssid', 'rssi']

def loadWifiLog(path):
    return (
        sc
        .textFile(path)
        .filter(lambda x: mac_address.search(x))
        .map(lambda x: x.split('\t'))
        .filter(lambda x: len(x) == len(cols))
        .toDF().toDF(*cols)
    )

wifi_log_files = glob.glob("./data/wifi-apc.*.log")

dfs = map(lambda x: loadWifiLog(x), wifi_log_files)

df = functools.reduce(lambda x, y: x.union(y), dfs)

dates = df.withColumn("date", to_date("timestamp")).select('date')
dates.distinct().orderBy("date").toPandas().head()

,date
0,2017-11-29
1,2017-11-30
2,2017-12-06
3,2017-12-07
4,2017-12-10


## GPS logs

Filter and concatenate data from GPS logs.

In [1]:
# .toDF('Timestamp','Level','Lat','Long','Altitude','Speed','Satellites','Mode','Time')